In [ ]:
import os, shutil
import fileinput as fi
import glob
import moviepy.editor as mp
import pandas as pd
import random
import time

os.chdir(path=r"D:\02_batch_2")
print("Current Working Directory " , os.getcwd())

df_dirname = []
for folder in sorted(os.listdir(os.getcwd())):
    df_dirname.append((folder))

df_dirname = pd.DataFrame(df_dirname, columns=['Folder'])
df_dirname.sort_values(by='Folder',inplace=True)
df_dirname

In [20]:
def as_perc(value, total):
    return value/float(total)

def aggregate_base(inPath,outPath):
    i = 0
    x = 0

    li = []
    path = inPath

    for i,dname in enumerate(df_dirname['Folder']):

        files = glob.glob(path + dname + '/*.txt')
        #print(f'Successfully created dataframe for {dname}')
        for i,f in enumerate(files):

            parse_error = False
            file_not_found = False

            stock = os.path.basename(f)            
            temp_df = pd.read_csv(f, delimiter="\t", error_bad_lines=False)
            temp_df['filePath'] = path + dname + '/'
            temp_df['textFileName'] = stock     
            temp_df['audioFileName'] = temp_df['textFileName'].replace('.txt', '.wav', regex=True)
            temp_df['duration'] = temp_df.stop-temp_df.start
            li.append(temp_df)    
    df = pd.concat(li, axis=0)

    df_percent = df.groupby(['filePath','textFileName']).agg({'duration':['sum']}).reset_index()
    df_percent=df_percent.droplevel(axis=1,level=1)
    df_percent.rename(columns={'duration':'duration_sum'},inplace=True)
    res = pd.DataFrame(df_percent.duration_sum.apply(as_perc, total=df_percent.duration_sum.sum()))
    df_percent['req_duration'] = round(res*180000)
    df_req_percent = df_percent.groupby('filePath').agg({'req_duration':'sum'}).reset_index()

    df_selection = pd.DataFrame(columns=['src_filepath','src_filepath_wav','duration']) 

    for i,fpath in enumerate(df_req_percent['filePath']):
        sum_duration = 0.0
        req_duration = df_req_percent.loc[i,'req_duration']
        filter_fpath = df_percent[df_percent['filePath'] == fpath]    

        for ifpath in filter_fpath['textFileName']:
            if (req_duration >= sum_duration):
                n = random.randint(0,len(filter_fpath)-1)                        
                df_selection.loc[x,'src_filepath'] = str(filter_fpath.iloc[n,0]) + str(filter_fpath.iloc[n,1])
                df_selection.loc[x,'src_filepath_wav'] = df_selection.loc[x,'src_filepath'].replace('.txt', '.wav')
                df_selection.loc[x,'duration'] = filter_fpath.iloc[n,2].astype('float32')
                sum_duration = sum_duration + filter_fpath.iloc[n,2].astype('float32')
                shutil.copy(df_selection.loc[x,'src_filepath'], outPath)
                shutil.copy(df_selection.loc[x,'src_filepath_wav'], outPath)
                x+=1  

    return df_selection


    

In [21]:
df_selection = aggregate_base (r'D:/02_batch_2/',r'D:/nf/audio/')
df_selection.head(3)

b'Skipping line 18: expected 5 fields, saw 7\n'
b'Skipping line 12: expected 5 fields, saw 6\n'
b'Skipping line 5: expected 5 fields, saw 8\n'


,src_filepath,src_filepath_wav,duration
0,D:/02_batch_2/37_degrees_series/37_degrees_ser...,D:/02_batch_2/37_degrees_series/37_degrees_ser...,184.290176
1,D:/02_batch_2/37_degrees_series/37_degrees_ser...,D:/02_batch_2/37_degrees_series/37_degrees_ser...,105.642891
2,D:/02_batch_2/37_degrees_series/37_degrees_ser...,D:/02_batch_2/37_degrees_series/37_degrees_ser...,147.067932


In [22]:
df_selection

,src_filepath,src_filepath_wav,duration
0,D:/02_batch_2/37_degrees_series/37_degrees_ser...,D:/02_batch_2/37_degrees_series/37_degrees_ser...,184.290176
1,D:/02_batch_2/37_degrees_series/37_degrees_ser...,D:/02_batch_2/37_degrees_series/37_degrees_ser...,105.642891
2,D:/02_batch_2/37_degrees_series/37_degrees_ser...,D:/02_batch_2/37_degrees_series/37_degrees_ser...,147.067932
3,D:/02_batch_2/37_degrees_series/37_degrees_ser...,D:/02_batch_2/37_degrees_series/37_degrees_ser...,171.292511
4,D:/02_batch_2/37_degrees_series/37_degrees_ser...,D:/02_batch_2/37_degrees_series/37_degrees_ser...,135.783447
...,...,...,...
794,D:/02_batch_2/zainab_almatroodi/zainab_almatro...,D:/02_batch_2/zainab_almatroodi/zainab_almatro...,279.390991
795,D:/02_batch_2/zainab_almatroodi/zainab_almatro...,D:/02_batch_2/zainab_almatroodi/zainab_almatro...,277.56601
796,D:/02_batch_2/zainab_almatroodi/zainab_almatro...,D:/02_batch_2/zainab_almatroodi/zainab_almatro...,293.903992
797,D:/02_batch_2/zainab_almatroodi/zainab_almatro...,D:/02_batch_2/zainab_almatroodi/zainab_almatro...,329.577057
